In [5]:
import sqlite3
import pandas as pd

db = sqlite3.connect('AMS.db')
query = 'select * from Order_Table'
df = pd.read_sql(query, con = db)
df['Date']=pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
dfMonth = df[(df['Year']==2019) & (df['Month']==12)]
dfOrderPL = dfMonth.groupby(['Date','OrderID'])[['SettleAmt']].sum()
dfOrderPL.reset_index()

,Date,OrderID,SettleAmt
0,2019-12-02,1,491.6
1,2019-12-02,2,101.6
2,2019-12-03,1,1691.6
3,2019-12-05,1,-564.4
4,2019-12-05,2,345.6
5,2019-12-05,3,95.6
6,2019-12-05,4,918.4
7,2019-12-09,1,975.6
8,2019-12-09,2,-774.4
9,2019-12-09,3,-554.4


In [2]:
dfMonth.groupby(['Date'])[['SettleAmt']].sum()

,SettleAmt
Date,
2019-12-02,593.2
2019-12-03,1691.6
2019-12-05,795.2
2019-12-09,-353.2
2019-12-10,171.2


In [3]:
df.groupby(['Year','Month'])[['SettleAmt']].sum()

SettleAmt
Year Month           
2019 11        2037.6
     12        2898.0

In [5]:
import portfolio
import pandas as pd

dt = '2019-12-23'
acct = 'CMS'

dfHold = portfolio.get_hold(dt, acct)
df = dfHold[['Cur','MV','PL']]
df_sum = df.groupby(['Cur']).sum()
df_sum

,MV,PL
Cur,,
CNY,125556.80,26890.40
HKD,108177.23,-7617.03


In [9]:
list(df_sum.index)

['CNY', 'HKD']

In [10]:
df_sum.loc['xxx'] = [0,0]
df_sum

,MV,PL
Cur,,
CNY,125556.80,26890.40
HKD,108177.23,-7617.03
xxx,0.00,0.00
